Part 2 of JAX TUT

Jax has a internal typing thing, will really help to assert things in code !https://jax.readthedocs.io/en/latest/jax.typing.html#module-jax.typing

In [4]:
import jax

array = jax.numpy.linspace(4, 10)
array.devices()

{CpuDevice(id=0)}

In [6]:
array.sharding

SingleDeviceSharding(device=CpuDevice(id=0), memory_kind=unpinned_host)

In [8]:
import jax.numpy as jnp

def selu(x, alpha=1.67, l=1.05):
    return l * jnp.where(x>0,x,alpha* jnp.exp(x) -alpha)

selu_jit = jax.jit(selu)
print(selu_jit(1.0))

1.05


Tracers extract the operations the array goes through when going through the transformations.

In [10]:
@jax.jit
def f(x):
    print(x)
    return x+1

x = jnp.arange(5)
print(f(x))

Traced<ShapedArray(int32[5])>with<DynamicJaxprTrace(level=1/0)>
[1 2 3 4 5]


Tracers help with finding the sequence of opersations. JAX then uses transfomations to obtain a representation of a functional projram

In [13]:
def selu(x, alpha=1.67, lambda_=1.05):
  return lambda_ * jnp.where(x > 0, x, alpha * jnp.exp(x) - alpha)

x = jnp.arange(5)
jax.make_jaxpr(selu)(x)

{ lambda ; a:i32[5]. let
    b:bool[5] = gt a 0
    c:f32[5] = convert_element_type[new_dtype=float32 weak_type=False] a
    d:f32[5] = exp c
    e:f32[5] = mul 1.6699999570846558 d
    f:f32[5] = sub e 1.6699999570846558
    g:f32[5] = pjit[
      name=_where
      jaxpr={ lambda ; h:bool[5] i:i32[5] j:f32[5]. let
          k:f32[5] = convert_element_type[new_dtype=float32 weak_type=False] i
          l:f32[5] = select_n h j k
        in (l,) }
    ] b a f
    m:f32[5] = mul 1.0499999523162842 g
  in (m,) }

JAX also uses trees to structure the chaotic informations of neural networks for example


In [ ]:
params = [1, 2, jnp.arange(4), (4, 2), jnp.ones(1)]

print(jax.tree.structure(params)) # Seems like a list is a pytree
print(jax.tree.structure((3, 2)))
print(jax.tree.structure(3))

print(jax.tree.leaves((3, ([32, 3], True))))

PyTreeDef([*, *, *, (*, *), *])
PyTreeDef((*, *))
PyTreeDef(*)
[3, 32, 3, True]


In [26]:
params = {"first": 4, "second": jnp.ones((3, 2)), "b": jnp.zeros(2)}

print(jax.tree.structure(params))
print(jax.tree.leaves(params))

# Named tuple of parameters
from typing import NamedTuple

class Params(NamedTuple):
  a: int
  b: float

params = Params(1, 5.0)
params.a

PyTreeDef({'b': *, 'first': *, 'second': *})
[Array([0., 0.], dtype=float32), 4, Array([[1., 1.],
       [1., 1.],
       [1., 1.]], dtype=float32)]


1